# Enhance ShopAssist AI (ShopAssist 2.0) - using Function Calling

Dataset obtained from : https://drive.google.com/file/d/1Z4Qta-ZpYGc-jjFstJXckyJKIvoQXTJj/view?usp=sharing

Starter Notebook obtained from: https://colab.research.google.com/drive/1T-a5WIcle8XhBS5aX5tzFbc2S3MwFnT1#scrollTo=ad9b47b8



# Objective:


* **Integrate Function Calling API:** Modify the existing architecture to leverage the Function Calling API's capabilities for improved performance. Scope out which layers can be removed and how the existing layers can be updated to handle the new approach.

* **Refine Conversation Flow:** Enhance the chatbot's conversation flow by leveraging function calling. Make the interaction between the user and the chatbot more natural and dynamic.

* **Document and Present:** Create comprehensive documentation that outlines the changes made, the integration process, and the benefits of using the Function Calling API. Prepare a presentation to showcase the enhancements.






# Import Libraries

In [25]:
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from google.colab import userdata
import pandas as pd

In [26]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Major functions behind the Chatbot - copy from starter notebook


- `initialize_conversation()`: This initializes the variable conversation with the system message.
- `get_chat_completions()`: This takes the ongoing conversation as the input and returns the response by the assistant
- `moderation_check()`: This checks if the user's or the assistant's message is inappropriate. If any of these is inappropriate, it ends the conversation.
- `intent_confirmation_layer()`: This function takes the assistant's response and evaluates if the chatbot has captured the user's profile clearly. Specifically, this checks if the following properties for the user has been captured or not GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget
- `dictionary_present()`: This function checks if the final understanding of user's profile is returned by the chatbot as a python dictionary or not. If there is a dictionary, it extracts the information as a Python dictionary.
- `compare_laptops_with_user()`: This function compares the user's profile with the different laptops and come back with the top 3 recommendations.
- `initialize_conv_reco()`: Initializes the recommendations conversation

In [27]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "low",
                        'Multitasking': "high",
                        'Processing speed': "high",
                        'Budget': "150000"}

    example_user_req = {'GPU intensity': "_",
                        'Display quality': "_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}

    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You need to ask relevant questions and understand the user profile by analysing the user's responses.
    You final objective is to fill the values for the different keys ('GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget') in the python dictionary and be confident of the values.
    These key value pairs define the user's profile.
    The python dictionary looks like this
    {{'GPU intensity': 'values','Display quality': 'values','Portability': 'values','Multitasking': 'values','Processing speed': 'values','Budget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    The values for all keys, except 'Budget', should be 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    All the values in the example dictionary are only representative values.
    {delimiter}
    Here are some instructions around the values for the different keys. If you do not follow this, you'll be heavily penalised:
    - The values for all keys, except 'Budget', should strictly be either 'low', 'medium', or 'high' based on the importance of the corresponding keys, as stated by user.
    - The value for 'Budget' should be a numerical value extracted from the user's response.
    - 'Budget' value needs to be greater than or equal to 25000 INR. If the user says less than that, please mention that there are no laptops in that range.
    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n
    {delimiter}
    Thought 1: Ask a question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask followup questions to understand their needs.
    You are trying to fill the values of all the keys {{'GPU intensity','Display quality','Portability','Multitasking','Processing speed','Budget'}} in the python dictionary by understanding the user requirements.
    Identify the keys for which you can fill the values confidently using the understanding. \n
    Remember the instructions around the values for the different keys.
    If the necessary information has been extracted, only then proceed to the next step. \n
    Otherwise, rephrase the question to capture their profile clearly. \n

    {delimiter}
    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}

    {delimiter}
    Here is a sample conversation between the user and assistant:
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    Assistant: "{example_user_dict}"
    {delimiter}

    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [28]:
def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    print(response)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [29]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    allowed_values = {'low','medium','high'}

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:
    {{
    'GPU intensity': 'values',
    'Display quality':'values',
    'Portability':'values',
    'Multitasking':'values',
    'Processing speed':'values',
    'Budget':'number'
    }}
    The values for the keys should only be from the allowed values: {allowed_values}.
    The 'Budget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.
    Only output a one-word string in JSON format at the key 'result' - Yes/No.
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    Thought 2 - If the answer is No, mention the reason in the key 'reason'.
    THought 3 - Think carefully before the answering.
    """

    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [30]:
def dictionary_present(response):
    delimiter = "####"

    user_req = {'GPU intensity': 'high',
                'Display quality': 'high',
                'Portability': 'medium',
                'Multitasking': 'high',
                'Processing speed': 'high',
                'Budget': '200000'}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:
            {{
            'GPU intensity': 'low/medium/high ',
            'Display quality':'low/medium/high',
            'Portability':'low/medium/high',
            'Multitasking':'low/medium/high',
            'Processing speed':'low/medium/high',
            'Budget':'numerical value'
            }}
            Here are some sample input output pairs for better understanding:
            {delimiter}
            input 1: - GPU intensity: low - Display quality: high - Portability: low - Multitasking: high - Processing speed: medium - Budget: 50,000 INR
            output 1: {{'GPU intensity': 'low', 'Display quality': 'high', 'Portability': 'low', 'Multitasking': 'high', 'Processing speed': 'medium', 'Budget': '50000'}}

            input 2: {{'GPU intensity':     'low', 'Display quality':     'low', 'Portability':    'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90,000'}}
            output 2: {{'GPU intensity': 'low', 'Display quality': 'low', 'Portability': 'medium', 'Multitasking': 'medium', 'Processing speed': 'low', 'Budget': '90000'}}

            input 3: Here is your user profile 'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000 INR'
            output 3: {{'GPU intensity': 'high','Display quality': 'high','Portability': 'medium','Multitasking': 'high','Processing speed': 'high','Budget': '200000'}}
            {delimiter}
            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [31]:
def product_map_layer(laptop_description):
    delimiter = "#####"

    lap_spec = {
        "GPU intensity":"(Type of the Graphics Processor)",
        "Display quality":"(Display Type, Screen Resolution, Display Size)",
        "Portability":"(Laptop Weight)",
        "Multitasking":"(RAM Size)",
        "Processing speed":"(CPU Type, Core, Clock Speed)"
    }

    values = {'low','medium','high'}

    prompt=f"""
    You are a Laptop Specifications Classifier whose job is to extract the key features of laptops and classify them as per their requirements.
    To analyze each laptop, perform the following steps:
    Step 1: Extract the laptop's primary features from the description {laptop_description}
    Step 2: Store the extracted features in {lap_spec} \
    Step 3: Classify each of the items in {lap_spec} into {values} based on the following rules: \
    {delimiter}
    GPU Intensity:
    - low: <<< if GPU is entry-level such as an integrated graphics processor or entry-level dedicated graphics like Intel UHD >>> , \n
    - medium: <<< if mid-range dedicated graphics like M1, AMD Radeon, Intel Iris >>> , \n
    - high: <<< high-end dedicated graphics like Nvidia RTX >>> , \n

    Display Quality:
    - low: <<< if resolution is below Full HD (e.g., 1366x768). >>> , \n
    - medium: <<< if Full HD resolution (1920x1080) or higher. >>> , \n
    - high: <<< if High-resolution display (e.g., 4K, Retina) with excellent color accuracy and features like HDR support. >>> \n

    Portability:
    - high: <<< if laptop weight is less than 1.51 kg >>> , \n
    - medium: <<< if laptop weight is between 1.51 kg and 2.51 kg >>> , \n
    - low: <<< if laptop weight is greater than 2.51 kg >>> \n

    Multitasking:
    - low: <<< If RAM size is 8 GB, 12 GB >>> , \n
    - medium: <<< if RAM size is 16 GB >>> , \n
    - high: <<< if RAM size is 32 GB, 64 GB >>> \n

    Processing Speed:
    - low: <<< if entry-level processors like Intel Core i3, AMD Ryzen 3 >>> , \n
    - medium: <<< if Mid-range processors like Intel Core i5, AMD Ryzen 5 >>> , \n
    - high: <<< if High-performance processors like Intel Core i7, AMD Ryzen 7 or higher >>> \n
    {delimiter}

    {delimiter}
    Here is input output pair for few-shot learning:
    input 1: "The Dell Inspiron is a versatile laptop that combines powerful performance and affordability. It features an Intel Core i5 processor clocked at 2.4 GHz, ensuring smooth multitasking and efficient computing. With 8GB of RAM and an SSD, it offers quick data access and ample storage capacity. The laptop sports a vibrant 15.6" LCD display with a resolution of 1920x1080, delivering crisp visuals and immersive viewing experience. Weighing just 2.5 kg, it is highly portable, making it ideal for on-the-go usage. Additionally, it boasts an Intel UHD GPU for decent graphical performance and a backlit keyboard for enhanced typing convenience. With a one-year warranty and a battery life of up to 6 hours, the Dell Inspiron is a reliable companion for work or entertainment. All these features are packed at an affordable price of 35,000, making it an excellent choice for budget-conscious users."
    output 1: {{'GPU intensity': 'medium','Display quality':'medium','Portability':'medium','Multitasking':'high','Processing speed':'medium'}}

    {delimiter}
    ### Strictly don't keep any other text in the values of the JSON dictionary other than low or medium or high ###
    """
    input = f"""Follow the above instructions step-by-step and output the dictionary in JSON format {lap_spec} for the following laptop {laptop_description}."""
    #see that we are using the Completion endpoint and not the Chatcompletion endpoint
    messages=[{"role": "system", "content":prompt },{"role": "user","content":input}]

    response = get_chat_completions(messages, json_format = True)

    #   response = openai.chat.completions.create(
    #     model="gpt-3.5-turbo-0125",
    #     messages=[{"role": "system", "content":prompt },{"role": "user","content":input}],
    #     response_format={ "type": "json_object" }
    #     # max_tokens = 2000,
    #     )

    # response = json.loads(response)
    return response

In [32]:
##extract product info
laptop_df= pd.read_csv('laptop_data.csv')

## Create a new column "laptop_feature" that contains the dictionary of the product features
laptop_df['laptop_feature'] = laptop_df['Description'].apply(lambda x: product_map_layer(x))

In [33]:
laptop_df.to_csv("updated_laptop.csv",index=False,header = True)

In [41]:
def compare_laptops_with_user(user_requirements):
    laptop_df = pd.read_csv('updated_laptop.csv')

    # Extracting the budget value from user_requirements and converting it to an integer
    budget = int(user_requirements.get('Budget', '0').replace(',', '').split()[0])
    # budget
    # # Creating a copy of the DataFrame and filtering laptops based on the budget
    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',', '').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    # filtered_laptops
    # # # Mapping string values 'low', 'medium', 'high' to numerical scores 0, 1, 2
    mappings = {'low': 0, 'medium': 1, 'high': 2}

    # # # Creating a new column 'Score' in the filtered DataFrame and initializing it to 0
    filtered_laptops['Score'] = 0

    # # # Iterating over each laptop in the filtered DataFrame to calculate scores based on user requirements
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = user_product_match_str
        laptop_values = dictionary_present(user_product_match_str)
        score = 0

    #     # Comparing user requirements with laptop features and updating scores
        for key, user_value in user_requirements.items():
            # if key.lower() == 'budget':
            if key == 'Budget':
                continue  # Skipping budget comparison
            laptop_value = laptop_values.get(key, None)
            # print(key, laptop_value)
            laptop_mapping = mappings.get(laptop_value, -1)
            # laptop_mapping = mappings.get(laptop_value, -1)
            # user_mapping = mappings.get(user_value, -1)
            user_mapping = mappings.get(user_value, -1)
            if laptop_mapping >= user_mapping:
                score += 1  # Incrementing score if laptop value meets or exceeds user value

        filtered_laptops.loc[index, 'Score'] = score  # Updating the 'Score' column in the DataFrame

    # Sorting laptops by score in descending order and selecting the top 3 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)
    top_laptops_json = top_laptops.to_json(orient='records')  # Converting the top laptops DataFrame to JSON format

    # top_laptops
    return top_laptops_json

In [35]:
def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1

In [36]:
def initialize_conv_reco(products):
    system_message = f"""
    You are an intelligent laptop gadget expert and you are tasked with the objective to \
    solve the user queries about any product from the catalogue in the user message \
    You should keep the user profile in mind while answering the questions.\

    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>

    """
    user_message = f""" These are the user's products: {products}"""
    conversation = [{"role": "system", "content": system_message },
                    {"role":"user","content":user_message}]
    # conversation_final = conversation[0]['content']
    return conversation

# Identify suitable functions for openai function calling


- **initialize_conversation**: Sets up the initial system message. Not a candidate for function calling.
- **get_chat_completions**: Interacts with OpenAI API. Not a candidate for function calling.
- **moderation_check**: Checks for inappropriate content. Could potentially be called by the model for safety checks, but less likely for core requirement processing.
- **intent_confirmation_layer**: Checks if user profile is captured. Takes assistant response and outputs a structured result (Yes/No and reason). *Good candidate for function calling.*
- **dictionary_present**: Extracts a dictionary from a string. Takes a string and outputs a dictionary. *Good candidate for function calling to parse model output.*
- **product_map_layer**: Extracts and classifies laptop features from descriptions. Takes a description and outputs a dictionary. *Good candidate for function calling to process product data.*
- **compare_laptops_with_user**: Compares user requirements with laptops and recommends top 3. Takes user requirements (string/dict) and outputs JSON of top laptops. *Good candidate for function calling to get recommendations.*
- **recommendation_validation**: Validates laptop recommendations based on a score threshold. Takes JSON of recommendations and outputs filtered list. *Good candidate for function calling to refine recommendations.*
- **initialize_conv_reco**: Initializes conversation for recommendations. Sets up system message. Not a candidate for function calling.

 Below Functions seem like strong candidates as they perform specific data processing tasks.

 - `intent_confirmation_layer`
 - `dictionary_present`
 - `product_map_layer`
 - `compare_laptops_with_user`
 - `recommendation_validation`



# Define function specifications for openai

Define the necessary function specifications (name, description, parameters, required parameters) in a format that OpenAI's API can understand for the identified functions.


In [37]:
import json

openai_functions = [
    {
        "name": "intent_confirmation_layer",
        "description": "Evaluates if the chatbot has clearly captured the user's profile based on specific criteria (GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget).",
        "parameters": {
            "type": "object",
            "properties": {
                "response_assistant": {
                    "type": "string",
                    "description": "The assistant's response containing the captured user requirements."
                }
            },
            "required": ["response_assistant"]
        }
    },
    {
        "name": "dictionary_present",
        "description": "Checks if a Python dictionary representing the user's profile is present in the input string and extracts it.",
        "parameters": {
            "type": "object",
            "properties": {
                "response": {
                    "type": "string",
                    "description": "The input string potentially containing the user's profile dictionary."
                }
            },
            "required": ["response"]
        }
    },
    {
        "name": "product_map_layer",
        "description": "Extracts and classifies key features (GPU intensity, Display quality, Portability, Multitasking, Processing speed) from a laptop description.",
        "parameters": {
            "type": "object",
            "properties": {
                "laptop_description": {
                    "type": "string",
                    "description": "The description of the laptop."
                }
            },
            "required": ["laptop_description"]
        }
    },
    {
        "name": "compare_laptops_with_user",
        "description": "Compares user requirements with available laptops, filters by budget, scores them based on matching features, and returns the top 3 recommendations as a JSON string.",
        "parameters": {
            "type": "object",
            "properties": {
                "user_req_string": {
                    "type": "string",
                    "description": "A string representation of the user's requirements dictionary. The function will internally parse this to a dictionary."
                }
            },
            "required": ["user_req_string"]
        }
    },
    {
        "name": "recommendation_validation",
        "description": "Validates a list of laptop recommendations (in JSON format) and filters them based on a minimum score threshold.",
        "parameters": {
            "type": "object",
            "properties": {
                "laptop_recommendation": {
                    "type": "string",
                    "description": "A JSON string representing the list of laptop recommendations, each with a 'Score' key."
                }
            },
            "required": ["laptop_recommendation"]
        }
    }
]

print(json.dumps(openai_functions, indent=2))

[
  {
    "name": "intent_confirmation_layer",
    "description": "Evaluates if the chatbot has clearly captured the user's profile based on specific criteria (GPU intensity, Display quality, Portability, Multitasking, Processing speed, Budget).",
    "parameters": {
      "type": "object",
      "properties": {
        "response_assistant": {
          "type": "string",
          "description": "The assistant's response containing the captured user requirements."
        }
      },
      "required": [
        "response_assistant"
      ]
    }
  },
  {
    "name": "dictionary_present",
    "description": "Checks if a Python dictionary representing the user's profile is present in the input string and extracts it.",
    "parameters": {
      "type": "object",
      "properties": {
        "response": {
          "type": "string",
          "description": "The input string potentially containing the user's profile dictionary."
        }
      },
      "required": [
        "response"
  

# Modify `get chat completions` to handle function calls

Modify the `get_chat_completions` function to include the defined function specifications and handle the responses from the OpenAI API when a function call is suggested. This involves parsing the function call request and executing the corresponding Python function.


In [38]:
import inspect

def get_chat_completions(input, json_format=False, functions=None):
    MODEL = 'gpt-3.5-turbo'

    messages = list(input) # Create a mutable copy of the input messages

    if json_format:
        system_message_json_output = """<<. Return output in JSON format to the key output.>>"""
        if messages and messages[0]['role'] == 'system':
            messages[0]['content'] += system_message_json_output
        else:
            messages.insert(0, {"role": "system", "content": system_message_json_output})


    try:
        if functions:
            chat_completion = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                functions=functions,
                function_call="auto",
                seed=1234 # Consistent seed for reproducibility
            )
        elif json_format:
             chat_completion = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                response_format={ "type": "json_object"},
                seed=1234 # Consistent seed for reproducibility
            )
        else:
            chat_completion = openai.chat.completions.create(
                model=MODEL,
                messages=messages,
                seed=2345 # Different seed when no function calling is involved
            )

        response_message = chat_completion.choices[0].message

        if response_message.function_call:
            function_name = response_message.function_call.name
            function_args = json.loads(response_message.function_call.arguments)

            # Map function names to function objects
            available_functions = {
                "intent_confirmation_layer": intent_confirmation_layer,
                "dictionary_present": dictionary_present,
                "product_map_layer": product_map_layer,
                "compare_laptops_with_user": compare_laptops_with_user,
                "recommendation_validation": recommendation_validation
            }

            if function_name in available_functions:
                function_to_call = available_functions[function_name]

                # Get function signature to check expected parameters
                sig = inspect.signature(function_to_call)
                params = sig.parameters.keys()

                # Filter function_args to only include parameters expected by the function
                filtered_args = {k: v for k, v in function_args.items() if k in params}

                # Check if all required parameters are present in filtered_args
                missing_required_params = [p for p, param in sig.parameters.items() if param.default == inspect.Parameter.empty and p not in filtered_args]

                if missing_required_params:
                    print(f"Error: Missing required parameters for function '{function_name}': {missing_required_params}")
                    # Handle this error appropriately, perhaps return an error message to the user
                    return "An error occurred while processing your request due to missing information for the function call."

                function_response = function_to_call(**filtered_args)

                # Append function call and response to messages
                messages.append(response_message) # the assistant's function_call message
                messages.append(
                    {
                        "role": "function",
                        "name": function_name,
                        "content": json.dumps(function_response) if isinstance(function_response, (dict, list)) else str(function_response),
                    }
                )

                # Make another API call with function output
                second_response = openai.chat.completions.create(
                    model=MODEL,
                    messages=messages,
                    seed=4567 # Different seed for the second call
                )
                return second_response.choices[0].message.content
            else:
                print(f"Error: Function '{function_name}' not found.")
                return "An error occurred: The requested function is not available."
        else:
            if json_format:
                return json.loads(response_message.content)
            else:
                return response_message.content

    except Exception as e:
        print(f"An error occurred during get_chat_completions: {e}")
        return "An error occurred while processing your request. Please try again."

# Integrate function calling into `dialogue mgmt system`

Modify the `dialogue_mgmt_system` function to use the updated `get_chat_completions` function and manage the conversation flow based on whether the model requests a function call or provides a regular message.


In [42]:
def dialogue_mgmt_system():
    global top_3_laptops

    conversation_history = initialize_conversation()

    # Initial welcome message
    welcome_message = get_chat_completions(conversation_history)
    display(welcome_message + '\n')

    top_3_laptops = None  # Initialize top_3_laptops to None

    user_input = ''

    while user_input.lower() != "exit":
        user_input = input("You: ")  # Prompt the user for their input

        moderation_status = moderation_check(user_input)
        if moderation_status == 'Flagged':
            display("Sorry, your message has been flagged. Please restart the conversation.")
            break

        # Append user input to the conversation history for processing
        conversation_history.append({"role": "user", "content": user_input})

        if top_3_laptops is None:
            # Gather initial response while determining user intent
            assistant_response = get_chat_completions(conversation_history, functions=openai_functions)

            if isinstance(assistant_response, str):  # Check if we received valid text response
                conversation_history.append({"role": "assistant", "content": assistant_response})
                print("Assistant:", assistant_response + '\n')

                # After the assistant provides the response, check if it contains the user requirements dictionary
                user_req_dict_str = dictionary_present(assistant_response)
                if user_req_dict_str:
                    try:
                        user_req_dict = json.loads(user_req_dict_str)
                        # Now call compare_laptops_with_user with the extracted dictionary
                        top_3_laptops = compare_laptops_with_user(user_req_dict)
                        print('\n' + "Products have been successfully fetched!" + '\n')
                        print("Here are some laptop recommendations based on your preferences:" + '\n')
                        # Initialize recommendation conversation
                        recommendation_conv = initialize_conv_reco(top_3_laptops)
                        # Continue the conversation with the recommendation summary
                        recommendation_summary = get_chat_completions(recommendation_conv)
                        print("Assistant:", recommendation_summary + '\n')

                    except json.JSONDecodeError:
                        print("Could not parse the user requirements dictionary from the assistant's response.")
                    except Exception as e:
                        print(f"An error occurred while getting recommendations: {e}")


            else:
                # Handle unexpected non-string responses gracefully
                print("Received an unexpected response:", assistant_response)

        else:
            # If in the recommendation phase, continue with follow-up questions
            follow_up_response = get_chat_completions(conversation_history)

            if isinstance(follow_up_response, str):
                conversation_history.append({"role": "assistant", "content": follow_up_response})
                print("Assistant:", follow_up_response + '\n')
            else:
                print("An issue occurred in the recommendation phase:", follow_up_response)

In [40]:
dialogue_mgmt_system()

'Hello! Welcome to the laptop recommendation assistant. Please share with me your requirements so that I can help you find the best laptop that suits your needs.\n'

You: I want a gaming laptop under 100000
ModerationCreateResponse(id='modr-BpY0eFTvvDRRffWgXNeiAz9YcyoQi', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=4.83349049318349e-06, harassment_threatening=2.344178756175097e-05, hate=9.738815833770786e-07, hate_threatening=1.3695890856979531e-06, illicit=None, illicit_violent=None, self_harm=0.0008077799575403333, self_harm_instructions=0.0006937571451999247, self_harm_intent=0.00757378526031971, sexual=2.223133924

KeyboardInterrupt: Interrupted by user

In [ ]:
dialogue_mgmt_system()

'Hello! Welcome to the laptop recommendation assistant. Please share with me your requirements so that I can help you find the best laptop that suits your needs.\n'

ModerationCreateResponse(id='modr-BpY9bfxGSHxbI9wH7Lao44uS9F8bw', model='text-moderation-007', results=[Moderation(categories=Categories(harassment=False, harassment_threatening=False, hate=False, hate_threatening=False, illicit=None, illicit_violent=None, self_harm=False, self_harm_instructions=False, self_harm_intent=False, sexual=False, sexual_minors=False, violence=False, violence_graphic=False, self-harm=False, sexual/minors=False, hate/threatening=False, violence/graphic=False, self-harm/intent=False, self-harm/instructions=False, harassment/threatening=False), category_applied_input_types=None, category_scores=CategoryScores(harassment=1.6026916682676529e-06, harassment_threatening=1.7772945284377784e-06, hate=3.287800609541591e-07, hate_threatening=1.1422570622698913e-07, illicit=None, illicit_violent=None, self_harm=5.633037289953791e-07, self_harm_instructions=9.663665878179017e-06, self_harm_intent=5.5395016715920065e-06, sexual=2.3979517209227197e-05, sexual_minors=2.957286